In [159]:
from langgraph.graph import StateGraph, START, END, add_messages
from typing import TypedDict, Annotated
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from jsonschema import validate
import json

In [160]:
load_dotenv()

True

In [ ]:
model = ChatGroq(model="llama-3.1-8b-instant")
# model = ChatOpenAI(model="gpt-4o-mini")

### Utility Functions

In [162]:
class SDLCState(TypedDict):
    user_input_requirements: str
    auto_generated_user_stories_markdown: Annotated[list[str], add_messages]

### Node Functions

In [163]:
def user_input_requirements(state: SDLCState):
    user_input = """
        I want to build a task management web application. 
        Features:
        - Users can create, update, and delete tasks.
        - Tasks should have priority and due dates.
        - A dashboard to track completed vs pending tasks.
        - Simple login with email and password.
        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.
        Deployment: Host on AWS.
        """
    
    return {"user_input_requirements": user_input}

In [164]:
def auto_generated_user_stories(state: SDLCState):
    prompt_template = PromptTemplate(
        template="""
         You are an expert Agile Product Owner and Business Analyst. 
         Your task is to generate high-quality, clear, and structured *user stories* from the given software requirements.  

         ### Input
         You will receive:
         - A description of user requirements (provided by the user).
         - The project context, if available.

         ### Instructions
         1. Break down the given requirements into **user stories** following the Agile format:  
            *As a [role], I want [feature] so that [benefit]*.
            
         2. Ensure that each user story:
            - Captures a single functionality or feature.  
            - Clearly identifies the **user role** (end-user, admin, developer, etc.).  
            - Describes the **action or feature** in simple language.  
            - States the **value or outcome** of the feature.  

         3. For each user story, provide:  
            - **User Story ID** (US-1, US-2, …)  
            - **Story Statement** (in Agile format)  
            - **Acceptance Criteria** (written as Given/When/Then or bullet points)  
            - **Priority** (High, Medium, Low)  
            - **Dependencies** (if any)  

         4. If requirements are ambiguous or incomplete:
            - Highlight unclear areas in a separate section called **Clarification Questions**.  

         ### Output Format
         Return the output in **well-structured Markdown** with the following sections:  
         - **User Stories** (list each with ID, statement, acceptance criteria, priority, dependencies)  
         - **Clarification Questions** (if needed)  

         ### Example
         **User Story (US-1):**  
         As a registered user, I want to reset my password so that I can regain access to my account when I forget it.  

         **Acceptance Criteria:**  
         - Given that the user is on the login page, when they click "Forgot Password", then they should be prompted to enter their email.  
         - An email with a reset link should be sent.  
         - The reset link should expire after 24 hours.  

         **Priority:** High  
         **Dependencies:** Email service setup  

         ---
         
         User Input: \n {user_input}
         
         ---

         Now, based on the provided requirements, generate a complete set of user stories.
   """,
        input_variables=["user_input"],
    )

    user_story_chain = prompt_template | model | StrOutputParser()

    markdown_response = user_story_chain.invoke(
        {"user_input": state["user_input_requirements"]}
    )

    return {"auto_generated_user_stories_markdown": markdown_response}

In [165]:
# THIS GRAPH IS USED FOR TESTING

graph = StateGraph(SDLCState)

graph.add_node("auto_generated_user_stories", auto_generated_user_stories)

graph.add_edge(START, "auto_generated_user_stories")
graph.add_edge("auto_generated_user_stories", END)

workflow = graph.compile()

In [166]:
response = workflow.invoke({"user_input_requirements": """
        I want to build a task management web application. 
        Features:
        - Users can create, update, and delete tasks.
        - Tasks should have priority and due dates.
        - A dashboard to track completed vs pending tasks.
        - Simple login with email and password.
        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.
        Deployment: Host on AWS.
        """})

In [167]:
response

{'user_input_requirements': '\n        I want to build a task management web application. \n        Features:\n        - Users can create, update, and delete tasks.\n        - Tasks should have priority and due dates.\n        - A dashboard to track completed vs pending tasks.\n        - Simple login with email and password.\n        Tech Stack Preference: Python (FastAPI) for backend, React for frontend.\n        Deployment: Host on AWS.\n        ',
 'auto_generated_user_stories_markdown': [HumanMessage(content='# User Stories\n\n**User Story (US-1):**  \nAs a user, I want to create tasks so that I can keep track of my responsibilities and manage my workload efficiently.\n\n**Acceptance Criteria:**  \n- Given that the user is logged in, when they navigate to the task creation page, then they should see a form to enter task details, including title, description, priority, and due date.  \n- The user should be able to submit the form to create a new task.  \n- Upon successful creation, 

In [168]:
def product_owner_review():
    pass

In [169]:
def create_design_docs():
    pass

In [170]:
def revise_user_stories():
    pass

In [171]:
def design_review():
    pass

In [172]:
def generate_code():
    pass

In [173]:
def code_review():
    pass

In [174]:
def fix_code_after_code_review():
    pass

In [175]:
def security_review():
    pass

In [176]:
def write_test_case():
    pass

In [177]:
def fix_code_after_security():
    pass

In [178]:
def test_case_review():
    pass

In [179]:
def qa_testing():
    pass

In [180]:
def fix_test_cases_after_review():
    pass

In [181]:
def deployment():
    pass

In [182]:
def fix_code_after_qa_feedback():
    pass

### Conditional Branches

In [183]:
def product_owner_response():
    response = "approved"
    if response == "approved":
        return response

In [184]:
def design_review_response():
    response = "approved"
    if response == "approved":
        return response

In [185]:
def code_review_response():
    response = "approved"
    if response == "approved":
        return response

In [186]:
def security_review_reponse():
    response = "approved"
    if response == "approved":
        return response

In [187]:
def test_case_review_response():
    response = "approved"
    if response == "approved":
        return response

In [188]:
def qa_testing_response():
    response = "passed"
    if response == "passed":
        return response

### Create Graph, Add nodes and edges

In [189]:
graph = StateGraph(SDLCState)

# Add Nodes
graph.add_node("user_input_requirements", user_input_requirements)
graph.add_node("auto_generated_user_stories", auto_generated_user_stories)
graph.add_node("product_owner_review", product_owner_review)
graph.add_node("create_design_docs", create_design_docs)
graph.add_node("revise_user_stories", revise_user_stories)
graph.add_node("design_review", design_review)
graph.add_node("generate_code", generate_code)
graph.add_node("code_review", code_review)
graph.add_node("fix_code_after_code_review", fix_code_after_code_review)
graph.add_node("security_review", security_review)
graph.add_node("write_test_case", write_test_case)
graph.add_node("fix_code_after_security", fix_code_after_security)
graph.add_node("test_case_review", test_case_review)
graph.add_node("qa_testing", qa_testing)
graph.add_node("fix_test_cases_after_review", fix_test_cases_after_review)
graph.add_node("deployment", deployment)
graph.add_node("fix_code_after_qa_feedback", fix_code_after_qa_feedback)


# Define Edges
graph.add_edge(START, "user_input_requirements")
graph.add_edge("user_input_requirements", "auto_generated_user_stories")
graph.add_edge("auto_generated_user_stories", "product_owner_review")
graph.add_conditional_edges(
    "product_owner_review",
    product_owner_response,
    {"approved": "create_design_docs", "feedback": "revise_user_stories"},
)
graph.add_edge(
    "revise_user_stories", "auto_generated_user_stories"
)  # When product_owner_response() is Feedback
graph.add_edge("create_design_docs", "design_review")

graph.add_conditional_edges(
    "design_review",
    design_review_response,
    {"approved": "generate_code", "feedback": "create_design_docs"},
)

graph.add_edge("generate_code", "code_review")
graph.add_conditional_edges(
    "code_review",
    code_review_response,
    {"approved": "security_review", "feedback": "fix_code_after_code_review"},
)
graph.add_edge("fix_code_after_code_review", "generate_code")

graph.add_conditional_edges(
    "security_review",
    security_review_reponse,
    {"approved": "write_test_case", "feedback": "fix_code_after_security"},
)
graph.add_edge("fix_code_after_security", "generate_code")

graph.add_edge("write_test_case", "test_case_review")
graph.add_conditional_edges(
    "test_case_review",
    test_case_review_response,
    {"approved": "qa_testing", "feedback": "fix_test_cases_after_review"},
)
graph.add_edge("fix_test_cases_after_review", "write_test_case")

graph.add_conditional_edges(
    "qa_testing",
    qa_testing_response,
    {"passed": "deployment", "failed": "fix_code_after_qa_feedback"},
)
graph.add_edge("fix_code_after_qa_feedback", "generate_code")

graph.add_edge("deployment", END)

In [190]:
# Compile the graph
workflow = graph.compile()

### Misc

In [191]:
"""
# Conditional Branches

1. product_owner_response()
2. design_review_response()
3. code_review_response()
4. security_review_reponse()
5. test_case_review_response()
6. qa_testing_response()
"""



"""
Nodes:

user_input_requirements()
auto_generated_user_stories()
product_owner_review()
create_design_docs() # Consist both functional & technical
revise_user_stories()
design_review()
generate_code()
code_review()
fix_code_after_code_review()
security_review()
write_test_case()
fix_code_after_security()
test_case_review()
qa_testing()
fix_test_cases_after_review()
deployment()
fix_code_after_qa_feedback()
"""

'\nNodes:\n\nuser_input_requirements()\nauto_generated_user_stories()\nproduct_owner_review()\ncreate_design_docs() # Consist both functional & technical\nrevise_user_stories()\ndesign_review()\ngenerate_code()\ncode_review()\nfix_code_after_code_review()\nsecurity_review()\nwrite_test_case()\nfix_code_after_security()\ntest_case_review()\nqa_testing()\nfix_test_cases_after_review()\ndeployment()\nfix_code_after_qa_feedback()\n'